In [ ]:
import os
import sys
import git
import pathlib

import random

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
os.environ['CUDA_VISIBLE_DEVICES'] = '0' # use GPU
# Using GPU during inference has deterministic results (same as CPU)

PROJ_ROOT_PATH = pathlib.Path(git.Repo('.', search_parent_directories=True).working_tree_dir)
PROJ_ROOT =  str(PROJ_ROOT_PATH)
if PROJ_ROOT not in sys.path:
    sys.path.append(PROJ_ROOT)

from libs import utils, mnist32_cnn
from libs.constants import model_seeds

In [ ]:
# Limit GPU growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [ ]:
model_type = "mnist32-cnn_1024_256_64"

In [ ]:
model_seed = model_seeds[0]

In [ ]:
dsfsd

In [ ]:
model_instance = model_type + "-" + str(seed)
dataset, model_arch, model_config, layer_widths, seed = utils.instancename2metadata(model_instance)
model_meta_type, model_type, model_instance = utils.metadata2instancenames(dataset, model_arch, layer_widths, seed)

model_folder = pathlib.Path(PROJ_ROOT_PATH / "models" / model_meta_type / model_type)
model_filename = model_instance + ".h5"
model_file = pathlib.Path(model_folder/ model_filename)

[accuracy, conf_matrix] = mnist32_cnn.test_mnist32(model_file, show_summary=False)
print(f"Model: {model_instance} \t Accuracy:{accuracy*100:0.3f}%")

In [ ]:
def test_mnist32(model_file, show_summary=False):

    # Prepare dataset
    # Combine test and train images together into one dataset
    DATASET_PATH = str(pathlib.Path(PROJ_ROOT_PATH / "datasets" / "mnist.npz" ))
    (train_images, train_labels), (test_images, test_labels) = mnist.load_data(path=DATASET_PATH)
    train_images = train_images.astype(np.float32) / 255.0
    test_images = test_images.astype(np.float32) / 255.0  

    all_images =np.concatenate([train_images, test_images], axis=0)
    all_labels =np.concatenate([train_labels, test_labels], axis=0)
    all_images = np.expand_dims(all_images, axis=-1)
    
    # resize the input shape , i.e. old shape: 28, new shape: 32
    image_x_size = 32
    image_y_size = 32
    all_images = tf.image.resize(all_images, [image_x_size, image_y_size]) 

    # Load model
    model = tf.keras.models.load_model(model_file)
    if show_summary:
        model.summary()

    # Evaluate model
    y_pred = model.predict(all_images) # Predict encoded label as 2 => [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
    Y_pred = np.argmax(y_pred, 1) # Decode Predicted labels

    accuracy = accuracy_score(y_true=all_labels, y_pred=Y_pred)
    conf_matrix = confusion_matrix(y_true=all_labels, y_pred=Y_pred) 
          
    return [accuracy, conf_matrix]

#############################################################################################